In [ ]:
from jax.scipy.special import ndtri
from matplotlib.pyplot import *
from jaxns import NestedSampler
from jaxns.prior_transforms import PriorChain, LogNormalPrior
from jaxns.prior_transforms.prior import prior_docstring, get_shape
from jaxns.prior_transforms.common import ContinuousPrior
from jax import random, numpy as jnp, jit
import numpy as np
import utils

## Draw samples from jaxns implimentation of the log-normal distribution

In [ ]:
def log_likelihood(p):
    
    return -200000
        
with PriorChain() as prior_chain:
    LogNormalPrior('p', 0, 1)
 
# Run the nested sampling
ns = NestedSampler(loglikelihood=log_likelihood, prior_chain=prior_chain)

f = jit(ns)

results = f(random.PRNGKey(3452345))

## Draw samples from an interpolated version of the log-normal quantile function

In [ ]:
class InterpPrior(ContinuousPrior):
    @prior_docstring
    def __init__(self, name, interp_func, tracked=True):
        """

        """
        self.interp_func = interp_func

        shape = []

        super(InterpPrior, self).__init__(name, shape, [], tracked)

    def transform_U(self, U, **kwargs):
        return self.interp_func(U)
    

x = np.linspace(0, 1, 10001)

y = jnp.exp(ndtri(x))

interp_func = utils.jaxInterp1D(x, y)    

with PriorChain() as prior_chainI:
    InterpPrior('p', interp_func)
    
nsI = NestedSampler(loglikelihood=log_likelihood, prior_chain=prior_chainI)

fI = jit(nsI)

resultsI = fI(random.PRNGKey(3452345))

## Plot results

In [ ]:
hist(results.samples['p'][~np.isinf(results.samples['p'])], bins = np.linspace(0,30,20), log=True, density=True)
hist(resultsI.samples['p'][~np.isinf(resultsI.samples['p'])], bins = np.linspace(0,30,20), log=True, density=True, alpha=0.5)
hist(np.exp(ndtri(np.random.uniform(0, 1, 10000))), bins = np.linspace(0,30,20), density=True, alpha = 0.5, histtype='step')